# URI Illegal Wildlife Trafficking: R workshop in network analysis

## Setup

This section installs and loads all the packages we need, and downloads a cleaned up version of the CITES dataset.  We'll call this dataset `cites_data`.

Click on the cell and press the play button, or press shift+enter to run the code

This takes approximately 10-15 minutes to run. We will return to the powerpoint introductions after you run this code.

In [ ]:
install.packages("igraph")
install.packages("googledrive")

library(igraph)
library(tidyverse)
library(googledrive)

# Collect dataset from google drive link
drive_deauth()
drive_download(as_id("https://drive.google.com/file/d/17sWWNYt05rLFUMCnQ-vlHG7cMmWUWqIm/view?usp%3Dsharing&sa=D&source=hangouts&ust=1618184626983000&usg=AFQjCNG-Gnc7UDyM6uoXgsUEVZZ7VBSCbw"), overwrite = TRUE)
cites_data <- read.csv('animals_clean.csv')

### Dataset details
The dataset was downloaded on April 10, 2021 using [pycites](https://github.com/WildlifeTradeNetworks/pycites), and additional taxonomical information was added based off of the [ITIS database](https://www.itis.gov/). 
Any row that had an `NA` for Year, Importer or Exporter was removed, and only items from after 1996 were included (when records "were checked for correct species distribution and valid combinations of taxon and term, taxon and source, etc" (see section 1.4 [here](https://trade.cites.org/cites_trade_guidelines/en-CITES_Trade_Database_Guide.pdf))). 
To reduce data size for this course, this dataset only includes trades involving Kingdom *Anamalia*.

## Activity 0: Test
Before we begin the class, let's make sure we can run some simple code and plot a graph. 
Click on each cell of code and press "shift + enter"

In [ ]:
# Anything in a coding cell that has a "#" in front is just a commeent about the code

# I am creating a variable x, that consists of 5 different numbers
x <- c(1,3,6,9,12)

# I am now creating a variable y, that consists of 5 different numbers
y <- c(1.5,2,7,8,15)

#I now want to plot x against y in a simple scatter plot 
plot(
    x, y, 
    xlab="x axis", 
    ylab="y axis", 
    main="My nice little plot with fairly large blue dots",
    ylim=c(0,20),
    xlim=c(0,15), 
    pch=19,
    col="blue",
    cex=3
)

If you see a simple scatter plot with 5 data points, you have successfully run R code!

##Activity 1: Examine CITES data

Now we'll start to explore the network, where each node is a country. See Annex 3 of the [CITES Trade Database Guide](https://trade.cites.org/cites_trade_guidelines/en-CITES_Trade_Database_Guide.pdf) to look up the country name from its 2-letter code.

### 1.1 Explore the structure (str) of the CITES dataset

In [ ]:
str(cites_data)

### 1.2 Print the first 6 rows of data

In [ ]:
head(cites_data)

### 1.3 Explore other properties of the dataset

In [ ]:
min(cites_data$Year) 

In [ ]:
max(cites_data$Year) 

In [ ]:
table(cites_data$itis_class) 

In [ ]:
sort(table(cites_data$itis_class))

In [ ]:
plot(sort(table(cites_data$itis_class)))

Answer the following questions: 

a. how many shipments does your data have?  
b. how many different classes are represented in your dataset?  
c. what timeframe does your dataset span? 


## Activity 2: Convert CITES data to network and visualize

##### 2.1 Run Functions

These functions will be used throughout this workshop to make things easier.

`create_network` makes an igraph network from a dataset, which we can use for further analysis and visualization.

`plot_network` allows us to plot an igraph network in a nicer way than by default, and easily change the edge weights

In [ ]:
create_network <- function(data){
    # create a tibble of unique Exporter and Importer countries
    # this is used for the network nodes
    sources <- data %>%
        distinct(Exporter) %>%
        rename(label = Exporter)  # all exporting countries (treated as country of origin)
    destinations <- data %>%
        distinct(Importer) %>%
        rename(label = Importer)  # all importing countries
    nodes <- full_join(sources, destinations, by = "label")  # all unique countries in your dataset 
    nodes <- nodes %>%
        rowid_to_column("id")  # create numerical id for each unique country 

    # create a tibble with export id, import id, and weight by number of shipments
    # this is used for the network edges
    per_route <- data %>%  
        group_by(Exporter, Importer) %>%
        summarise(weight = n()) %>% #number of observations in the current group
        ungroup() #removes grouping 
    edges <- per_route %>% 
        left_join(nodes, by = c("Exporter" = "label")) %>% 
        rename(from = id)
    edges <- edges %>% 
        left_join(nodes, by = c("Importer" = "label")) %>% 
        rename(to = id)
    edges <- select(edges, from, to, weight)

    # create a directed igraph network from the edges and nodes defined above   
    net <- graph_from_data_frame(d=edges, vertices=nodes, directed=T)
    return(net)
}


plot_network <- function(
    igraph_network,
    title="trade network",
    weight_factor=0.01, # scaling factor for edge weights
    layout=layout_with_mds  # igraph layour
    ){
    E(igraph_network)$width <- E(igraph_network)$weight*weight_factor
    l <- layout(igraph_network)
    plot(
        # TODO: adjust plot size, etc to make look niceer in Colab
        igraph_network,
        edge.color= rgb(70/255, 130/255, 180/255, 0.6),
        edge.arrow.size=.001, 
        vertex.size = 10, 
        vertex.color= rgb(255/255,190/255,0/255, 0.8),
        vertex.label = V(igraph_network)$id,
        vertex.label.cex = 0.6, 
        vertex.label.color = "black", 
        rescale=T, 
        layout=l*3.0, 
        main=title) 
}

"cell run successfully"

### 2.2 Create and visualize network

Now we'll start to explore the network, where each node is a country. See Annex 3 of the [CITES Trade Database Guide](https://trade.cites.org/cites_trade_guidelines/en-CITES_Trade_Database_Guide.pdf) to look up the country name from its 2-letter code.

In [ ]:
cites_data_network <-create_network(cites_data) 
'cell run successfully'

In [ ]:
plot_network(cites_data_network, weight_factor=0.00002, layout = layout_randomly)

What an uninformative mess! This doesn't really tell us anything. 
We'll remove some nodes (such as Unknown or blank country names), to clean up our dataset.

In [ ]:
# get vertex ID of ambiguous country codes
unknown <- which(V(cites_data_network)$label == "XX")
various <- which(V(cites_data_network)$label == "XV")
blank <- which(V(cites_data_network)$label == "")
# delete them from our network
cites_data_network <- delete_vertices(cites_data_network, c(unknown, blank, various))
'cell run successfully'

Next, we'll remove nodes with weights less than 1000, so we can see the major nodes in our network.

In [ ]:
s1 <- subgraph.edges(cites_data_network, E(cites_data_network)[E(cites_data_network)$weight>1000], del=F)
s2 <- delete_vertices(s1, degree(s1, mode = "in")==0)
plot_network(s2, weight_factor=0.00001, layout = layout_randomly)

Much better! You can right click on the figure above and open in a new tab to zoom in

There are many different network layouts in igraph. Let's try some of them yourself

In [ ]:
plot_network(s2, weight_factor=0.00001, layout = layout_on_sphere)

In [ ]:
plot_network(s2, weight_factor=0.00001, layout = layout_with_mds)

In [ ]:
plot_network(s2, weight_factor=0.00001, layout = layout.circle)


In [ ]:
plot_network(s2, weight_factor=0.00001, layout = layout_with_kk)


In [ ]:
plot_network(s2, weight_factor=0.00001, layout = layout_with_dh)


In [ ]:
plot_network(s2, weight_factor=0.00001, layout = layout_on_grid)

Which layouts do you like? And why? Is there anything else about this graph that you would like to see? 

Type in your answer here: 

## Activity 3: Compare subsets of Amphibian data

### 3.1 Subset 2 datasets, convert to network, and visualize

In [ ]:
Amp_1996 <- subset(cites_data, itis_class == "Amphibia" & Year == "1996")  # Create subset of Amphibia data from 1996
Amp_2016 <- subset(cites_data, itis_class == "Amphibia" & Year == "2016")  # Create subset of Amphibia data from 2016
'cell run successfully'

In [ ]:
# Create networks from these subsets
net1 <-create_network(Amp_1996)
net2 <-create_network(Amp_2016)
'cell run successfully'

In [ ]:
par(mfrow=c(1,2), mar = c(0.1, 0.1, 1, 0.1))  # create a graph with 1 row and 2 columns of plots
plot_network(net1, title="Amphibians 1996", weight_factor=0.1, layout= layout_on_sphere)
plot_network(net2, title="Amphibians 2016", weight_factor=0.1, layout= layout_on_sphere)

### 3.2 Visual cluster analysis of subsets

In [ ]:
par(mfrow=c(1,2), mar = c(0.1, 0.1, 1, 0.1))  # create a graph with 1 row and 2 columns of plots
cfg <- cluster_fast_greedy(as.undirected(net1))  # run clustering algorithm on our first network
plot(cfg, as.undirected(net1)) 
title(main = "Amphibians 1996")
cfg <- cluster_fast_greedy(as.undirected(net2)) # run clustering algorithm on our second network
plot(cfg, as.undirected(net2)) 
title(main = "Amphibians 2016")

Compare the clusters from 1996 and 2016. Each color represents a module, or a clique of countries that are similar to each other. How are the cliques different between 1996 and 2016? What do you think is happening to the amphibian trade over time? 

Write your answer here:

## Activity 4: Network and node descriptive stats

### 4.1 Examine network metrics for Amphibian 1996 network

Visually inspecting your network can be fun, but it doesn't give you a quantitative description of your data. For example, if you have a straight trendline through an x-y scattr plot, it is still useful to find out numerical properties like the mean, variance, slope of regression line, intercept, etc. 

For the next few examples, we are only going to examine the properties of net1, which is the amphibian trade in 1996

**Density:** The proportion of present edges from all possible ties.

In [ ]:
ecount(net1)/(vcount(net1)*(vcount(net1)-1)) #for a directed network

**Reciprocity:** The proportion of reciprocated ties (for a directed network).

In [ ]:
reciprocity(net1)

**Global Transitivity:** ratio of triangles (direction disregarded) to connected triples

In [ ]:
transitivity(net1, type="global")

**Diameter:** longest geodesic distance
Note that edge weights are used by default, unless set to NA.

In [ ]:
diameter(net1, directed=F, weights=NA)

**Node degree:** Number of connections each node has to other nodes'

In [ ]:
deg <- degree(net1, mode="all") #NODE METRICS ************
plot(net1, vertex.size=deg*2, edge.arrow.size=0.1)
hist(deg, breaks=1:vcount(net1)-1, main="Histogram of node degree")

**Hubs:** lots of outgoing links


In [ ]:
hs <- hub_score(net1, weights=NA)$vector
plot(net1, vertex.size=hs*50, main="Hubs", edge.arrow.size=0.1, vertex.color= rgb(212/255, 175/255, 55/255, 0.6))

**Authorities:** lots of incoming links from hubs

In [ ]:
as <- authority_score(net1, weights=NA)$vector
plot(net1, vertex.size=as*30, main="Authorities", edge.arrow.size=0.1, vertex.color= rgb(212/255, 175/255, 55/255, 0.6)) 

**Community detection**

In [ ]:
clp <- cluster_label_prop(net1)
V(net1)$community <- clp$membership
plot(net1, vertex.color=V(net1)$community)

We are going to consolidate all the network metrics from activity 4 above into one cell, so it's easier to compare with the next section of code.

In [ ]:
density_1996 <- ecount(net1)/(vcount(net1)*(vcount(net1)-1)) # density
reciprocity_1996 <- reciprocity(net1)
transityivity_1996 <- transitivity(net1, type="global")
diameter_1996 <- diameter(net1, directed=F, weights=NA)
deg_1996 <- degree(net1, mode="all") 
hs_1996 <- hub_score(net1, weights=NA)$vector
as_1996 <- authority_score(net1, weights=NA)$vector

### 4.2 Examine network metrics for Amphibian 2016 network

You have just successfully examined network metrics for the 1996 Amphibian network (net1). Now, do the same thing for the 2016 Amphibian network (net2)

**Density**

In [ ]:
density_2016 <- ecount(net2)/(vcount(net2)*(vcount(net2)-1)) # density
paste("1996: ", density_1996)
paste("2016: ", density_2016)

**Reprocity**

In [ ]:
reciprocity_2016 <- reciprocity(net2)
paste("1996: ", reciprocity_1996)
paste("2016: ", reciprocity_2016)

**Transitivity**

In [ ]:
transityivity_2016 <- transitivity(net2, type="global")
paste("1996: ", transityivity_1996)
paste("2016: ", transityivity_2016)

**Diameter**

In [ ]:
diameter_2016 <- diameter(net2, directed=F, weights=NA)
paste("1996: ", diameter_1996)
paste("2016: ", diameter_2016)

**Node degree**

In [ ]:
deg_2016 <- degree(net2, mode="all")

par(mfrow=c(1,2), mar = c(0.1, 0.1, 1, 0.1))  # create a graph with 1 row and 2 columns of plots
plot(net1, vertex.size=deg_1996, edge.arrow.size=0.1, main="1996")
plot(net2, vertex.size=deg_2016, edge.arrow.size=0.1, main="2016")

In [ ]:
hist(deg_1996, breaks=1:vcount(net2)-1, main="1996")
hist(deg_2016, breaks=1:vcount(net2)-1, main="2016")

**Hubs**

In [ ]:
hs_2016 <- hub_score(net2, weights=NA)$vector

par(mfrow=c(1,2), mar = c(0.1, 0.1, 1, 0.1))  # create a graph with 1 row and 2 columns of plots
plot(net1, vertex.size=hs_1996*30, main="Hubs 1996", edge.arrow.size=0.1, vertex.color= rgb(212/255, 175/255, 55/255, 0.6))
plot(net2, vertex.size=hs_2016*30, main="Hubs 2016", edge.arrow.size=0.1, vertex.color= rgb(212/255, 175/255, 55/255, 0.6))

**Authorities**

In [ ]:
as_2016 <- authority_score(net2, weights=NA)$vector

par(mfrow=c(1,2), mar = c(0.1, 0.1, 1, 0.1))  # create a graph with 1 row and 2 columns of plots
plot(net1, vertex.size=as_1996*30, main="Authorities 1996", edge.arrow.size=0.1, vertex.color= rgb(212/255, 175/255, 55/255, 0.6)) 
plot(net2, vertex.size=as_2016*30, main="Authorities 2016", edge.arrow.size=0.1, vertex.color= rgb(212/255, 175/255, 55/255, 0.6)) 

How are the metrics from 1996 Amphibians (net1) and 2016 (net2) different? 

Type in your answer here:

## Activity 5: If we have time, compare network metrics with two different subsets of your choice

*You can pick any two logical subsets to compare, or choose one of the following suggested pairs. Make sure you actually have data by examining the structure of your subsets! hint:
table(yoursubset$Year) to see how many entries it has*

Don't use entries later than 2016 as data is incomplete


Option 1: Insects in 1996 vs Insects in 2015

Option 2: African elephants vs Indian elephants

Option 3: Tortoises in the 1990s vs Tortoises in the 2000s 



In [ ]:
###############################################
#Option 1: Insects in 1996 vs Insects in 2015
###############################################
Insect1996 <- subset(cites_data, itis_class == "Insecta" & Year == "1996")  # Create subset of Insect data from 1996
Insect2015 <- subset(cites_data, itis_class == "Insecta" & Year == "2015")  # Create subset of Insect data from 2015
table(Insect1996$Year)
table(Insect2015$Year)
Ins1 <-create_network(Insect1996) #Create network from 1996 insect data 
Ins2 <-create_network(Insect2015) #Create network from 2015 insect data 
par(mfrow=c(1,2), mar = c(0.1, 0.1, 1, 0.1))  # create a graph with 1 row and 2 columns of plots
plot_network(Ins1, title="Insects 1996", weight_factor=0.1, layout= layout_on_sphere) #plot graph 
plot_network(Ins2, title="Insects 2015", weight_factor=0.1, layout= layout_on_sphere) # plot graph

What's the difference between the Insect trade in the two years? Number of nodes? Number of shipments? 

Type your answer here:

###5.1 modify the insect code above to answer your own question

For those of you who want to code by yourself, ignore my code below and copy and paste the code above and modify

If you prefer to run my code, let's look at 2 more examples below! 

In [ ]:
###############################################
#Option 2: African elephants vs Indian elephants
###############################################

African <- subset(cites_data, Genus == "Loxodonta")
Indian <- subset (cites_data, Genus== "Elephas")
table(African$Year)
table(Indian$Year)
Loxnet <-create_network(African) 
Elephasnet <-create_network(Indian) 
s1 <- subgraph.edges(Loxnet, E(Loxnet)[E(Loxnet)$weight>10], del=F)
Loxnet <- delete_vertices(s1, degree(s1, mode = "in")==0)
s2 <- subgraph.edges(Elephasnet, E(Elephasnet)[E(Elephasnet)$weight>10], del=F)
Elephasnet <- delete_vertices(s2, degree(s2, mode = "in")==0)
par(mfrow=c(1,2), mar = c(0.1, 0.1, 1, 0.1))  # create a graph with 1 row and 2 columns of plots
plot_network(Loxnet, title="African Elephants", weight_factor=0.1, layout= layout_on_sphere) #plot graph 
plot_network(Elephasnet, title="Indian Elepahtns", weight_factor=0.1, layout= layout_on_sphere) # plot graph

In [ ]:
###############################################
#Option 3: Tortoises in the 1990s vs Tortoises in the 2000s 
###############################################
Torts90s <- subset(cites_data, Family == "Testudinidae" & Year > 1989 & Year < 2000 )
Torts20s <- subset(cites_data, Family == "Testudinidae" & Year > 1999 & Year < 2010 )
table(Torts90s$Year)
table(Torts20s$Year)
Tortnet1 <-create_network(Torts90s)
Tortnet2 <-create_network(Torts20s)
s1 <- subgraph.edges(Tortnet1, E(Tortnet1)[E(Tortnet1)$weight>10], del=F)
Tortnet1 <- delete_vertices(s1, degree(s1, mode = "in")==0)
s2 <- subgraph.edges(Tortnet2, E(Tortnet2)[E(Tortnet2)$weight>10], del=F)
Tortnet2 <- delete_vertices(s2, degree(s2, mode = "in")==0)
par(mfrow=c(1,2), mar = c(0.1, 0.1, 1, 0.1))  # create a graph with 1 row and 2 columns of plots
plot_network(Tortnet1, title="Tortoises 90s", weight_factor=0.02, layout= layout_on_sphere)
plot_network(Tortnet2, title="Tortoises 2000s", weight_factor=0.02, layout= layout_on_sphere)

## References and Further Reading
Csardi G, Nepusz T: The igraph software package for complex network research, InterJournal, Complex Systems 1695. 2006.
  http://igraph.sf.net

Kolaczyk, E.D. and Csárdi, G., 2014. Statistical analysis of network data with R (Vol. 65). New York, NY: Springer.

Luke, D.A., 2015. A user's guide to network analysis in R. Cham, Switzerland: Springer.

Ognyanova, K. (2016) Network analysis with R and igraph: NetSci X Tutorial. Retrieved from www.kateto.net/networks-r-igraph.

Ognyanova, K. (2019) Network visualization with R. Retrieved from www.kateto.net/network-visualization.

Ross, Noam, Evan A. Eskew, and Nicolas Ray. 2019. citesdb: A high-performance database of shipment-level CITES trade data. R package v0.2.0. EcoHealth Alliance: New York, NY. https://github.com/ropensci/citesdb. doi:10.5281/zenodo.2630836

UNEP-WCMC (Comps.) 2019. Full CITES Trade Database Download. Version 2019.2. CITES Secretariat, Geneva, Switzerland. Compiled by UNEP-WCMC, Cambridge, UK. Available at: https://trade.cites.org.

